## Download data from NASA POWER

In [8]:
import requests
import pandas as pd
from itertools import product
from time import sleep

# === Define grid (urban area) ===
lat_min, lat_max = -12.3, -12.15
lon_min, lon_max = -39.03, -38.95
step = 0.05  # around 5-6 points per axis, total ~36 points

# === Helper to iterate float ranges ===
def frange(start, stop, step):
    while start <= stop:
        yield start
        start += step


lats = [round(x, 4) for x in frange(lat_min, lat_max, step)]
lons = [round(x, 4) for x in frange(lon_min, lon_max, step)]
points = list(product(lats, lons))


# === Parameters for monthly request ===
base_url = "https://power.larc.nasa.gov/api/temporal/monthly/point"
params_template = {
    "start": "2010",          # ✅ valid
    "end": "2024",            # ✅ valid
    "community": "AG",
    "format": "JSON",
    "parameters": ",".join([
        "PRECTOTCORR", "RH2M", "QV2M",
        "GWETROOT", "GWETPROF", "GWETTOP", "CLOUD_AMT"
    ])
}

# === Collect data for each point ===
all_data = []

for i, (lat, lon) in enumerate(points):
    print(f"[{i+1}/{len(points)}] Fetching lat={lat}, lon={lon}...")
    params = params_template.copy()
    params["latitude"] = lat
    params["longitude"] = lon

    try:
        r = requests.get(base_url, params=params)
        r.raise_for_status()
        raw = r.json()
        time_series = raw["properties"]["parameter"]
        all_months = list(time_series.values())[0].keys()
        for date in all_months:
            if "-999" in str([time_series[v].get(date) for v in time_series]):
                continue
            row = {
                "lat": lat,
                "lon": lon,
                "year": int(date[:4]),
                "month": int(date[4:])
            }
            for var in time_series:
                row[var] = time_series[var][date]
            all_data.append(row)
        sleep(1)  # avoid rate limiting
    except Exception as e:
        print(f"⚠️ Error fetching {lat},{lon}: {e}")
        continue

# === Save all data ===
df = pd.DataFrame(all_data)
df.to_csv("flood_features.csv", index=False)
print("✅ Done! CSV saved with shape:", df.shape)


[1/6] Fetching lat=-12.3, lon=-39.03...
[2/6] Fetching lat=-12.3, lon=-38.98...
[3/6] Fetching lat=-12.25, lon=-39.03...
[4/6] Fetching lat=-12.25, lon=-38.98...
[5/6] Fetching lat=-12.2, lon=-39.03...
[6/6] Fetching lat=-12.2, lon=-38.98...
✅ Done! CSV saved with shape: (1128, 11)
